In [1]:


import sys, subprocess, importlib, os, copy, shutil
from pathlib import Path
from dataclasses import dataclass
from typing import List, Optional, Tuple

def ensure_packages(packages: List[str]) -> None:
    """
    Intenta importar cada paquete y, si falta, lo instala con pip.
    En tu Jupyter local funcionará (requiere internet).
    """
    for pkg in packages:
        try:
            importlib.import_module(pkg)
        except ImportError:
            try:
                print(f"Instalando {pkg}...")
                subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
            except Exception as e:
                print(f"⚠️ No se pudo instalar {pkg}. Error: {e}")


PREC = {'|': 2, '·': 3, '?': 4, '*': 4, '+': 4, '^': 5, '(': 1}
RIGHT_ASSOC = {'^'} 
@dataclass
class PasoSY:
    paso: int
    token: str
    accion: str
    pila: str
    salida: str

@dataclass
class ResultadoSY:
    linea: int
    original: str
    postfix: str
    postfix_tokens: List[str]
    error_msg: Optional[str]
    pasos: List[PasoSY]

def es_operador(tok: str) -> bool:
    return tok in PREC and tok != '('

def tokenizar(regex: str) -> List[str]:
   
    tokens: List[str] = []
    i = 0
    while i < len(regex):
        c = regex[i]
        if c == '\\' and i + 1 < len(regex):
            tokens.append(regex[i:i+2])
            i += 2
        else:
            if not c.isspace():
                tokens.append(c)
            i += 1
    return tokens

def necesita_concat(c1: str, c2: str) -> bool:
  
    if c1 == '(' or c2 == ')':
        return False
    if c2 == '|':
        return False
    if c2 in {'?', '*', '+'}:
        return False
    if es_operador(c1) and PREC[c1] != 4:  # 4 = ?,*,+ (postfijos)
        return False
    return True

def format_regex(regex: str) -> List[str]:
    """Inserta el operador de concatenación '·' donde corresponda."""
    toks = tokenizar(regex)
    if not toks:
        return []
    res: List[str] = []
    for i, t1 in enumerate(toks[:-1]):
        t2 = toks[i+1]
        res.append(t1)
        c1 = t1[-1]
        c2 = t2[0]
        if necesita_concat(c1, c2):
            res.append('·')
    res.append(toks[-1])
    return res

def infix_to_postfix(regex: str, idx_linea: int = 1) -> ResultadoSY:
    tokens = format_regex(regex)
    salida: List[str] = []
    pila: List[str] = []
    pasos: List[PasoSY] = []
    step = 1
    error: Optional[str] = None

    def push(tok: str):
        nonlocal step
        pila.append(tok)
        pasos.append(PasoSY(step, tok, 'push', ''.join(pila), ''.join(salida))); step += 1

    def pop_to_output():
        nonlocal step
        tok = pila.pop()
        salida.append(tok)
        pasos.append(PasoSY(step, tok, 'pop→out', ''.join(pila), ''.join(salida))); step += 1

    for tok in tokens:
        if error: break
        if tok == '(':
            push(tok)
        elif tok == ')':
            if not pila:
                error = "Paréntesis de cierre sin apertura."
                pasos.append(PasoSY(step, tok, 'error', ''.join(pila), ''.join(salida))); step += 1
                break
            while pila and pila[-1] != '(':
                pop_to_output()
            if not pila or pila[-1] != '(':
                error = "Paréntesis de cierre sin apertura."
                pasos.append(PasoSY(step, tok, 'error', ''.join(pila), ''.join(salida))); step += 1
                break
            pila.pop()
            pasos.append(PasoSY(step, ')', 'pop (', ''.join(pila), ''.join(salida))); step += 1
        elif es_operador(tok):
            while (pila and es_operador(pila[-1]) and
                   ((PREC[pila[-1]] > PREC[tok]) or
                    (PREC[pila[-1]] == PREC[tok] and tok not in RIGHT_ASSOC))):
                pop_to_output()
            push(tok)
        else:
            salida.append(tok)
            pasos.append(PasoSY(step, tok, 'output', ''.join(pila), ''.join(salida))); step += 1

    if not error:
        while pila and pila[-1] != '(':
            pop_to_output()
        if pila and pila[-1] == '(':
            error = "Paréntesis de apertura sin cierre."
            pasos.append(PasoSY(step, '', 'error', ''.join(pila), ''.join(salida))); step += 1

    postfix_tokens = salida[:]
    postfix_str = ''.join(salida)
    return ResultadoSY(idx_linea, regex, postfix_str, postfix_tokens, error, pasos)


def desugar_postfix_extensions(postfix_tokens: List[str]) -> List[str]:
 
    stack: List[List[str]] = []
    for tok in postfix_tokens:
        if tok == '·':
            if len(stack) < 2: raise ValueError("Postfix inválido: faltan operandos para '·'.")
            b = stack.pop(); a = stack.pop()
            stack.append(a + b + ['·'])
        elif tok == '|':
            if len(stack) < 2: raise ValueError("Postfix inválido: faltan operandos para '|'.")
            b = stack.pop(); a = stack.pop()
            stack.append(a + b + ['|'])
        elif tok == '*':
            if not stack: raise ValueError("Postfix inválido: falta operando para '*'.")
            a = stack.pop()
            stack.append(a + ['*'])
        elif tok == '+':
            if not stack: raise ValueError("Postfix inválido: falta operando para '+'.")
            a = stack.pop()
            stack.append(a + a + ['*', '·'])  # X+ -> X X * ·
        elif tok == '?':
            if not stack: raise ValueError("Postfix inválido: falta operando para '?'.")
            a = stack.pop()
            stack.append(a + ['ε', '|'])      # X? -> X ε |
        else:
            stack.append([tok])
    if len(stack) != 1:
        raise ValueError("Postfix inválido: sobran elementos tras la desazucaración.")
    return stack[0]


@dataclass
class ASTNode: ...
@dataclass
class Epsilon(ASTNode):
    def label(self) -> str: return "ε"
@dataclass
class Literal(ASTNode):
    value: str
    def label(self) -> str:
        return self.value[1] if len(self.value) == 2 and self.value[0] == '\\' else self.value
@dataclass
class Concat(ASTNode):
    left: ASTNode; right: ASTNode
    def label(self) -> str: return "·"
@dataclass
class UnionNode(ASTNode):
    left: ASTNode; right: ASTNode
    def label(self) -> str: return "|"
@dataclass
class Star(ASTNode):
    child: ASTNode
    def label(self) -> str: return "*"

def is_epsilon_token(tok: str) -> bool:
    return tok in {"ε", "ϵ", "𝜀"}

def postfix_to_ast(postfix_tokens: List[str]) -> ASTNode:
    
    stack: List[ASTNode] = []
    for tok in postfix_tokens:
        if tok == '·':
            if len(stack) < 2: raise ValueError("Faltan operandos para concatenación.")
            b = stack.pop(); a = stack.pop()
            stack.append(Concat(a, b))
        elif tok == '|':
            if len(stack) < 2: raise ValueError("Faltan operandos para unión.")
            b = stack.pop(); a = stack.pop()
            stack.append(UnionNode(a, b))
        elif tok == '*':
            if not stack: raise ValueError("Falta operando para kleene *.") 
            x = stack.pop()
            stack.append(Star(x))
        else:
            stack.append(Epsilon() if is_epsilon_token(tok) else Literal(tok))
    if len(stack) != 1: raise ValueError("Postfix inválido: pila no quedó con 1 elemento.")
    return stack[0]


def _collect_edges(node: ASTNode, start_id: int = 0):
    edges: List[Tuple[int,int]] = []
    labels: List[Tuple[int,str]] = []
    nid = start_id
    def visit(n: ASTNode) -> int:
        nonlocal nid
        my = nid; nid += 1
        if   isinstance(n, Epsilon):   labels.append((my, n.label()))
        elif isinstance(n, Literal):   labels.append((my, n.label()))
        elif isinstance(n, Concat):
            labels.append((my, n.label()))
            l = visit(n.left); r = visit(n.right)
            edges.append((my, l)); edges.append((my, r))
        elif isinstance(n, UnionNode):
            labels.append((my, n.label()))
            l = visit(n.left); r = visit(n.right)
            edges.append((my, l)); edges.append((my, r))
        elif isinstance(n, Star):
            labels.append((my, n.label()))
            c = visit(n.child)
            edges.append((my, c))
        else:
            labels.append((my, str(type(n).__name__)))
        return my
    root = visit(node)
    return edges, labels, nid

def ast_to_dot(node: ASTNode, title: Optional[str] = None) -> str:
    edges, labels, _ = _collect_edges(node)
    lines = ["digraph AST {", '  rankdir=TB;']
    if title:
        lines.append(f'  labelloc="t"; label="{title}"; fontsize=20;')
    for nid, lab in labels:
        safe = lab.replace('"', r'\"')
        lines.append(f'  {nid} [label="{safe}"];')
    for u, v in edges:
        lines.append(f'  {u} -> {v};')
    lines.append("}")
    return "\n".join(lines)

def draw_ast(node: ASTNode, titulo: Optional[str], output_path: Optional[str], formato: str = "png"):

    dot_src = ast_to_dot(node, titulo)
    out_png = None
    out_dot = None
    if output_path:
        out_dot = f"{output_path}.dot"
        with open(out_dot, "w", encoding="utf-8") as f:
            f.write(dot_src)
        print(f"📝 DOT guardado: {out_dot}")
        
        try:
            ensure_packages(["graphviz"])
            import graphviz  # type: ignore
            g = graphviz.Source(dot_src, format=formato)
            out_png = g.render(filename=output_path, cleanup=True)
            print(f"✅ Imagen renderizada con Graphviz: {out_png}")
        except Exception as e:
            print(f"⚠️ Graphviz no disponible o falló: {e}")
            # Fallback
            try:
                ensure_packages(["networkx", "matplotlib"])
                import networkx as nx  # type: ignore
                import matplotlib.pyplot as plt  # type: ignore
                edges, labels, _ = _collect_edges(node)
                G = nx.DiGraph()
                for nid, lab in labels:
                    G.add_node(nid, label=lab)
                G.add_edges_from(edges)
                try:
                    pos = nx.nx_agraph.graphviz_layout(G, prog="dot")
                except Exception:
                    pos = nx.spring_layout(G, seed=42)
                plt.figure(figsize=(8, 6))
                nx.draw(G, pos, with_labels=False)
                nx.draw_networkx_labels(G, pos, labels={nid: lab for nid, lab in labels})
                out_png = f"{output_path}.png"
                plt.savefig(out_png, bbox_inches="tight")
                plt.close()
                print(f"✅ Imagen guardada con NetworkX/Matplotlib: {out_png}")
            except Exception as e2:
                print(f"⚠️ Tampoco se pudo con NetworkX/Matplotlib: {e2}")
    return out_dot, out_png


def run_lab(input_path: str,
            out_dir: str = "./arboles",
            formato: str = "png",
            show_simplified_postfix: bool = True,
            zip_output: bool = True):

    ruta = Path(input_path)
    if not ruta.exists():
        print(f"❌ No existe el archivo: {ruta}")
        return None
    os.makedirs(out_dir, exist_ok=True)

    results = []
    for idx, linea in enumerate(ruta.read_text(encoding='utf-8').splitlines(), 1):
        expr = (linea or "").strip()
        if not expr:
            continue
        print("\n" + "═"*60)
        print(f"Línea {idx}: {expr}")
        res = infix_to_postfix(expr, idx_linea=idx)
        if res.error_msg:
            print(f"❌ Error Shunting Yard: {res.error_msg}")
            continue
        try:
            simple = desugar_postfix_extensions(res.postfix_tokens)
        except Exception as e:
            print(f"❌ Error desazucarando postfix: {e}")
            continue
        if show_simplified_postfix:
            print(f"Postfix (simplificado): {simple}  ->  '{''.join(simple)}'")
        try:
            ast = postfix_to_ast(simple)
        except Exception as e:
            print(f"❌ Error construyendo AST: {e}")
            continue
        titulo = f"Árbol AST (Línea {idx})"
        out_path = os.path.join(out_dir, f"linea_{idx}")
        out_dot, out_png = draw_ast(ast, titulo, out_path, formato=formato)
        results.append((idx, expr, simple, out_dot, out_png))

    zip_path = None
    if zip_output:
        zip_path = shutil.make_archive(out_dir, 'zip', out_dir)
        print(f"📦 ZIP creado: {zip_path}")
    return results, zip_path


def crear_archivo_ejemplo(path: str = "expresiones.txt") -> str:
    ejemplos = [
        "(a*|b*)+",
        "((ε|a)|b*)*",
        "(a|b)*abb(a|b)*",
        "0?(1?)?0*"
    ]
    with open(path, "w", encoding="utf-8") as f:
        for e in ejemplos:
            f.write(e + "\n")
    print(f"🗂️ Archivo de ejemplo creado en: {path}")
    return path


run_lab("expresiones.txt", out_dir="./arboles", formato="png", zip_output=True)




════════════════════════════════════════════════════════════
Línea 1: (a*|b*)+
Postfix (simplificado): ['a', '*', 'b', '*', '|', 'a', '*', 'b', '*', '|', '*', '·']  ->  'a*b*|a*b*|*·'
📝 DOT guardado: ./arboles/linea_1.dot
✅ Imagen renderizada con Graphviz: arboles/linea_1.png

════════════════════════════════════════════════════════════
Línea 2: ((ε|a)|b*)*
Postfix (simplificado): ['ε', 'a', '|', 'b', '*', '|', '*']  ->  'εa|b*|*'
📝 DOT guardado: ./arboles/linea_2.dot
✅ Imagen renderizada con Graphviz: arboles/linea_2.png

════════════════════════════════════════════════════════════
Línea 3: (a|b)*abb(a|b)*
Postfix (simplificado): ['a', 'b', '|', '*', 'a', '·', 'b', '·', 'b', '·', 'a', 'b', '|', '*', '·']  ->  'ab|*a·b·b·ab|*·'
📝 DOT guardado: ./arboles/linea_3.dot
✅ Imagen renderizada con Graphviz: arboles/linea_3.png

════════════════════════════════════════════════════════════
Línea 4: 0?(1?)?0*
Postfix (simplificado): ['0', 'ε', '|', '1', 'ε', '|', 'ε', '|', '·', '0', '*', '·']  -

([(1,
   '(a*|b*)+',
   ['a', '*', 'b', '*', '|', 'a', '*', 'b', '*', '|', '*', '·'],
   './arboles/linea_1.dot',
   'arboles/linea_1.png'),
  (2,
   '((ε|a)|b*)*',
   ['ε', 'a', '|', 'b', '*', '|', '*'],
   './arboles/linea_2.dot',
   'arboles/linea_2.png'),
  (3,
   '(a|b)*abb(a|b)*',
   ['a', 'b', '|', '*', 'a', '·', 'b', '·', 'b', '·', 'a', 'b', '|', '*', '·'],
   './arboles/linea_3.dot',
   'arboles/linea_3.png'),
  (4,
   '0?(1?)?0*',
   ['0', 'ε', '|', '1', 'ε', '|', 'ε', '|', '·', '0', '*', '·'],
   './arboles/linea_4.dot',
   'arboles/linea_4.png')],
 '/Users/javiervalladares/Library/Mobile Documents/com~apple~CloudDocs/Sexto semestre/teoría de la compu/Lab3T/LAb3T/arboles.zip')

In [4]:
# === Ejecutar con tu archivo expresiones.txt ===
from pathlib import Path

# Ruta al archivo con las expresiones (una por línea)
exprs_file = Path("expresiones.txt")

if not exprs_file.exists():
    raise FileNotFoundError(f"No se encontró {exprs_file.resolve()}. "
                            "Asegúrate de que 'expresiones.txt' esté en el mismo directorio del notebook.")

# Cadena w a simular contra TODAS las expresiones del archivo (cámbiala si querés)
try:
    w = input("Ingresa la cadena w a simular (ENTER para 'abb'): ").strip() or "abb"
except Exception:
    w = "abb"

# Ejecuta el pipeline: infix -> postfix -> desazúcar -> AST -> AFN (Thompson) -> dibujo + simulación

    expressions_path=str(exprs_file),
    w=w,
    out_dir="./afn",        # carpeta de salida para .dot/.png
    formato="png",
    zip_output=True,        # genera ./afn.zip con todos los AFN
    trace_simulacion=True   # muestra el recorrido de la simulación

